# Exploratory Data Analysis та навчання моделі

Процес EDA та навчання моделі розпочнемо із імпортування необхідних для подальшої роботи бібліотек. 

In [17]:
import math
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime
from sklearn.datasets import load_boston
from sklearn.datasets import make_regression
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
import matplotlib.pyplot as plt
from sklearn import set_config

Далі зчитаємо дані з наданого датасету. Поле *install_date* зчитаємо як дату, враховуючи, що у датасеті дати подані у форматі *yyyy-mm-dd*. 

In [ ]:
dateparse = lambda dates: [datetime.strptime(d, '%Y-%m-%d') for d in dates]
df = pd.read_csv('dataset.zip', parse_dates=['install_date'], date_parser=dateparse)

Далі отримаємо новий датасет із старого, позбувшись входжень, які містять **nan** та порівняємо довжину старого та отриманого датасетів. 

In [11]:
df_without_nans = df.dropna()
print("Original dataset length: ", len(df))
print("Dataset without nans length: ", len(df_without_nans))
print("Difference: ", len(df) - len(df_without_nans))

Original dataset length:  1467832
Dataset without nan's length:  1466458
Difference:  1374


Враховуючи розмір датасету, можемо знехтувати даними, які містять значення **nan** і працювати із новим датасетом.

In [12]:
df = df_without_nans

Поля, що містять категоріальний тип даних перетворимо на числові. Для цього позбудемося приставок SOURCE_ та COUNTRY_ у відповідних полях, отриманий результат переведемо в цілочисельний тип, що дасть нам номер кожного сорсу та країни, а дані у полі *install_date* зведемо до цілочисельного типу після перетворення до типу *timestamp*. Поле *platform* зробимо бінарним, залишивши 0 в разі, якщо платформа користувача - *iOS* і 1 - *android*.

In [13]:
df['media_source'] = df['media_source'].apply(lambda t: int(t.replace("SOURCE_", "")))
df['install_date'] = df['install_date'].apply(lambda t: int(t.timestamp()))
df['country_code'] = df['country_code'].apply(lambda t: int(t.replace("COUNTRY_", "")))
df['platform'] = df['platform'].apply(lambda t: 0 if t == 'ios' else 1)

Оберемо поля, які будуть застосовуватися для передбачення та цільову змінну (в нашому випадку це *target_full_ltv_day30*).

In [14]:
x = df.iloc [:, : -4]
y = np.ravel(df.iloc [:, -1 :], order='C')

Оскільки вище були отримані цілочисельні значення полів медіасорс та кантрі, то виникає проблема інтерпретації вихідних чисел, оскільки усі ресурси та країни не мають вносити різний внесок у результат в залежності від їхнього індексу. Для вирішення цього конфлікту стандартизуємо дані.

In [15]:
x = scale(x)
y = scale(y)

Розділимо тепер дані на тренувальні та тестові. Для передбачення застосуємо алгоритм *random forest regression*.

In [16]:
xtrain, xtest, ytrain, ytest=train_test_split(x, y, test_size=0.2)

set_config(print_changed_only=False)

regressor = RandomForestRegressor(n_estimators=100)

regressor.fit(xtrain, ytrain)

KeyboardInterrupt: 

Перевіримо отриману модель на тестових даних та отримаємо метрики R2, MAE, MSE, MAPE та RMSE і виведемо їх на екран.

In [ ]:
ypred = regressor.predict(xtest)

mae = mean_absolute_error(ytest, ypred)
mse = mean_squared_error(ytest, ypred)
mape = mean_absolute_percentage_error(ytest, ypred)
rmse = math.sqrt(mse)

print("MAE: ", mae)
print("MSE: ", mse)
print("RMSE: ", rmse)
print("MAPE: ", mape)